# Handwritten numbers Recognition

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras 

## Load Data

Extraer datos. Las imágenes son de 28*28 píxeles, por lo tanto el csv tiene 784 columnas más 1 indicando la salida

In [11]:
def extract():
    df_train = pd.read_csv('data/mnist_train.csv', delimiter = ',').to_numpy()
    df_test = pd.read_csv('data/mnist_test.csv', delimiter = ',').to_numpy()
    df = pd.DataFrame(np.concatenate((df_train,df_test)))
    return df

Transformar output a array de 10 y separar de las inputs

In [12]:
def transform(df): 
    y_raw = df.iloc[:, 0].to_numpy()
    x = df.drop(df.columns[0], axis='columns').to_numpy()
    y = []
    for i in range(0,y_raw.shape[0]):
        row=np.zeros(10)
        row[y_raw[i]]=1
        y.append(row)
    y=np.array(y)
    y.reshape(y.shape[0],10)
    return x, y

Creamos los dataset de entrenamiento y test

In [13]:
df = extract()
x_train,y_train = transform(df.iloc[0:60000])
#x_val,y_val = transform(df.iloc[50000:60000])
x_test,y_test = transform(df.iloc[60000:70000])

## Creación de la Red Neuronal

En la red neuronal vamoas a implementar una arquitectura de (784,512,256,128,10), con funciones de activación ReLu en todas las capas menos en la última, en la cual utilizaremos la funcion softmax (https://www.quora.com/Why-is-it-better-to-use-Softmax-function-than-sigmoid-function).<br> Para cada capa se utilizará la funcion Dense, la cual tiene los siguientes parámetros por defecto:
- activation=None,
- use_bias=True,
- kernel_initializer='glorot_uniform',
- bias_initializer='zeros',
- kernel_regularizer=None,
- bias_regularizer=None,
- activity_regularizer=None,
- kernel_constraint=None,
- bias_constraint=None




In [14]:
model = keras.Sequential([
    keras.layers.Dense(784, input_dim=784),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax")#Softmax is a sort of Multi Class Sigmoid
])

Vamos a crear y entrenar el modelo, 50 epochs serán suficientes.

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
model.fit(x_train, y_train, epochs = 50, batch_size=200, verbose=1)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 3s 57us/sample - loss: 4.1446 - accuracy: 0.8754
Epoch 2/50
60000/60000 [==============================] - 3s 43us/sample - loss: 0.6380 - accuracy: 0.9305
Epoch 3/50
60000/60000 [==============================] - 2s 40us/sample - loss: 0.4266 - accuracy: 0.9437
Epoch 4/50
60000/60000 [==============================] - 3s 43us/sample - loss: 0.3458 - accuracy: 0.9516
Epoch 5/50
60000/60000 [==============================] - 3s 50us/sample - loss: 0.2710 - accuracy: 0.9593
Epoch 6/50
60000/60000 [==============================] - 3s 51us/sample - loss: 0.2502 - accuracy: 0.9600
Epoch 7/50
60000/60000 [==============================] - 3s 51us/sample - loss: 0.2462 - accuracy: 0.9617
Epoch 8/50
60000/60000 [==============================] - 3s 50us/sample - loss: 0.2159 - accuracy: 0.9649
Epoch 9/50
60000/60000 [==============================] - 3s 50us/sample - loss: 0.1946 - accuracy: 0.9656
Epoch 10/50
60

Evaluamos el modelo con el acierto de los datos de prueba

In [16]:
tes_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 73us/sample - loss: 0.1853 - accuracy: 0.9632
Test Acc:  0.96319264


## Regularización

In [17]:
from keras import regularizers
modelR = keras.Sequential([
    keras.layers.Dense(784, input_dim=784, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01))
])

In [18]:
modelR.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
modelR.fit(x_train, y_train, epochs = 50, batch_size=200, verbose=1)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 4s 65us/sample - loss: 12.9482 - accuracy: 0.8739
Epoch 2/50
60000/60000 [==============================] - 3s 58us/sample - loss: 7.7219 - accuracy: 0.9249
Epoch 3/50
60000/60000 [==============================] - 3s 58us/sample - loss: 5.5973 - accuracy: 0.9449
Epoch 4/50
60000/60000 [==============================] - 4s 59us/sample - loss: 4.0585 - accuracy: 0.9545
Epoch 5/50
60000/60000 [==============================] - 4s 64us/sample - loss: 2.9203 - accuracy: 0.9627
Epoch 6/50
60000/60000 [==============================] - 4s 64us/sample - loss: 2.0880 - accuracy: 0.9672
Epoch 7/50
60000/60000 [==============================] - 4s 68us/sample - loss: 1.4903 - accuracy: 0.9693
Epoch 8/50
60000/60000 [==============================] - 4s 71us/sample - loss: 1.0712 - accuracy: 0.9705
Epoch 9/50
60000/60000 [==============================] - 3s 57us/sample - loss: 0.7909 - accuracy: 0.9704
Epoch 10/50
6

In [19]:
tes_loss, test_acc = modelR.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 101us/sample - loss: 0.3183 - accuracy: 0.9616
Test Acc:  0.9615923


## Arquitectura [784-512-256-128-10]

In [20]:
from keras import regularizers
modelR = keras.Sequential([
    keras.layers.Dense(784, input_dim=784, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01))
])
modelR.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
modelR.fit(x_train, y_train, epochs = 50, batch_size=200, verbose=1)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 5s 87us/sample - loss: 17.1156 - accuracy: 0.8864
Epoch 2/50
60000/60000 [==============================] - 5s 83us/sample - loss: 10.3716 - accuracy: 0.9534
Epoch 3/50
60000/60000 [==============================] - 5s 80us/sample - loss: 7.0937 - accuracy: 0.9659
Epoch 4/50
60000/60000 [==============================] - 5s 80us/sample - loss: 4.8548 - accuracy: 0.9730
Epoch 5/50
60000/60000 [==============================] - 5s 80us/sample - loss: 3.3377 - accuracy: 0.9758
Epoch 6/50
60000/60000 [==============================] - 5s 81us/sample - loss: 2.3171 - accuracy: 0.9763
Epoch 7/50
60000/60000 [==============================] - 5s 82us/sample - loss: 1.6430 - accuracy: 0.9737s - los
Epoch 8/50
60000/60000 [==============================] - 5s 80us/sample - loss: 1.1962 - accuracy: 0.9718
Epoch 9/50
60000/60000 [==============================] - 5s 80us/sample - loss: 0.9005 - accuracy: 0.9718
Epoch

In [21]:
tes_loss, test_acc = modelR.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 115us/sample - loss: 0.3238 - accuracy: 0.9607
Test Acc:  0.96069217


## Arquitectura [784-512-128-10]

In [22]:
from keras import regularizers
modelR = keras.Sequential([
    keras.layers.Dense(784, input_dim=784, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01))
])
modelR.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
modelR.fit(x_train, y_train, epochs = 50, batch_size=200, verbose=1)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 5s 86us/sample - loss: 16.6130 - accuracy: 0.83952s - loss: 20.4251 - accuracy: 0 - ETA: 2s 
Epoch 2/50
60000/60000 [==============================] - 4s 75us/sample - loss: 10.1101 - accuracy: 0.9158TA: 0s - loss: 10.1828 - accuracy: 0
Epoch 3/50
60000/60000 [==============================] - 4s 73us/sample - loss: 7.6835 - accuracy: 0.9356
Epoch 4/50
60000/60000 [==============================] - 4s 73us/sample - loss: 5.7943 - accuracy: 0.9517
Epoch 5/50
60000/60000 [==============================] - 4s 74us/sample - loss: 4.3459 - accuracy: 0.9596s
Epoch 6/50
60000/60000 [==============================] - 4s 74us/sample - loss: 3.2334 - accuracy: 0.9654
Epoch 7/50
60000/60000 [==============================] - 4s 73us/sample - loss: 2.3884 - accuracy: 0.9702
Epoch 8/50
60000/60000 [==============================] - 4s 72us/sample - loss: 1.7643 - accuracy: 0.9719
Epoch 9/50
60000/60000 [===============

In [23]:
tes_loss, test_acc = modelR.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 110us/sample - loss: 0.2609 - accuracy: 0.9675
Test Acc:  0.9674935


## Arquitectura [784-128-10]

In [26]:
from keras import regularizers
modelR = keras.Sequential([
    keras.layers.Dense(784, input_dim=784, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01))
])
modelR.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
modelR.fit(x_train, y_train, epochs = 50, batch_size=200, verbose=1)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 3s 42us/sample - loss: 11.3336 - accuracy: 0.7432
Epoch 2/50
60000/60000 [==============================] - 2s 37us/sample - loss: 6.4254 - accuracy: 0.8495
Epoch 3/50
60000/60000 [==============================] - 2s 37us/sample - loss: 5.0784 - accuracy: 0.8758
Epoch 4/50
60000/60000 [==============================] - 2s 37us/sample - loss: 4.0511 - accuracy: 0.8884
Epoch 5/50
60000/60000 [==============================] - 2s 41us/sample - loss: 3.1976 - accuracy: 0.9003
Epoch 6/50
60000/60000 [==============================] - 3s 47us/sample - loss: 2.5042 - accuracy: 0.9109
Epoch 7/50
60000/60000 [==============================] - 3s 46us/sample - loss: 1.9197 - accuracy: 0.9223
Epoch 8/50
60000/60000 [==============================] - 3s 46us/sample - loss: 1.4514 - accuracy: 0.9348
Epoch 9/50
60000/60000 [==============================] - 3s 46us/sample - loss: 1.1066 - accuracy: 0.9383
Epoch 10/50
6

In [27]:
tes_loss, test_acc = modelR.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 83us/sample - loss: 0.2516 - accuracy: 0.9601
Test Acc:  0.960092


## Arquitectura [784-512-10]

In [28]:
from keras import regularizers
modelR = keras.Sequential([
    keras.layers.Dense(784, input_dim=784, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01))
])
modelR.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
modelR.fit(x_train, y_train, epochs = 50, batch_size=200, verbose=1)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 4s 62us/sample - loss: 16.2291 - accuracy: 0.8923
Epoch 2/50
60000/60000 [==============================] - 3s 57us/sample - loss: 9.3030 - accuracy: 0.9303
Epoch 3/50
60000/60000 [==============================] - 3s 56us/sample - loss: 7.2086 - accuracy: 0.9466
Epoch 4/50
60000/60000 [==============================] - 3s 56us/sample - loss: 5.5493 - accuracy: 0.9564
Epoch 5/50
60000/60000 [==============================] - 4s 66us/sample - loss: 4.2184 - accuracy: 0.9619
Epoch 6/50
60000/60000 [==============================] - 4s 71us/sample - loss: 3.1695 - accuracy: 0.9642
Epoch 7/50
60000/60000 [==============================] - 4s 71us/sample - loss: 2.3526 - accuracy: 0.9676
Epoch 8/50
60000/60000 [==============================] - 4s 72us/sample - loss: 1.7373 - accuracy: 0.9677
Epoch 9/50
60000/60000 [==============================] - 4s 71us/sample - loss: 1.2696 - accuracy: 0.9695
Epoch 10/50
6

In [29]:
tes_loss, test_acc = modelR.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 101us/sample - loss: 0.2843 - accuracy: 0.9603
Test Acc:  0.96029204


## Arquitectura [784-256-10]

In [31]:
from keras import regularizers
modelR = keras.Sequential([
    keras.layers.Dense(784, input_dim=784, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax", kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01))
])
modelR.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#learning_rate=0.001
modelR.fit(x_train, y_train, epochs = 50, batch_size=200, verbose=1)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 3s 55us/sample - loss: 14.2053 - accuracy: 0.8289
Epoch 2/50
60000/60000 [==============================] - 3s 55us/sample - loss: 7.7531 - accuracy: 0.8982
Epoch 3/50
60000/60000 [==============================] - 3s 55us/sample - loss: 6.2434 - accuracy: 0.9227
Epoch 4/50
60000/60000 [==============================] - 3s 55us/sample - loss: 5.0292 - accuracy: 0.9336
Epoch 5/50
60000/60000 [==============================] - 3s 55us/sample - loss: 3.9958 - accuracy: 0.9452
Epoch 6/50
60000/60000 [==============================] - 3s 56us/sample - loss: 3.1650 - accuracy: 0.9481
Epoch 7/50
60000/60000 [==============================] - 3s 56us/sample - loss: 2.4624 - accuracy: 0.9545
Epoch 8/50
60000/60000 [==============================] - 3s 55us/sample - loss: 1.8925 - accuracy: 0.9582
Epoch 9/50
60000/60000 [==============================] - 3s 55us/sample - loss: 1.4493 - accuracy: 0.9588
Epoch 10/50
6

In [32]:
tes_loss, test_acc = modelR.evaluate(x_test, y_test)
print('Test Acc: ', test_acc)

9998/9998 [==============================] - 1s 90us/sample - loss: 0.6168 - accuracy: 0.9106
Test Acc:  0.9105821
